In this notebook I have added code to join three datasets together and write it as

 json file so we can build another set of code to proces the json file in a production scenario.


dbfs:/FileStore/orders_csv.csv



 dbfs:/FileStore/customersweek17.csv

 dbfs:/FileStore/order_item.csv

In [0]:
orders_schema = "order_id long, order_date , customer_id long, order_status string"

In [0]:
orders_df = spark.read \
  .format("csv") \
    .schema(orders_schema) \
      .option("header",True) \
            .load("dbfs:/FileStore/orders_csv.csv")

In [0]:
orders_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)



In [0]:

orders_df.createOrReplaceTempView("orders")

In [0]:


from pyspark.sql.types import *

order_item_schema = StructType([
    StructField("order_item_id", LongType(), True),
    StructField("order_item_order_id", LongType(), True),
    StructField("order_item_product_id", LongType(), True),
    StructField("order_item_quantity", LongType(), True),
    StructField("order_item_subtotal", DecimalType(10, 2), True),  # Adjust precision & scale as needed
    StructField("order_item_product_price", DecimalType(10, 2), True)
])

In [0]:
order_items_df = spark.read \
    .schema(order_item_schema) \
        .format("csv") \
            .option("header",True) \
                .load("dbfs:/FileStore/order_item.csv")

In [0]:
order_items_df.createOrReplaceTempView("order_items")

In [0]:
spark.sql("select * from order_items").show()

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|            1|              36460|                  707|                  4|            1540.32|                  385.08|
|            2|              13154|                   26|                  2|             583.70|                  291.85|
|            3|              43230|                  828|                  2|             957.34|                  478.67|
|            4|              61127|                    6|                  4|            1656.36|                  414.09|
|            5|              59984|                  224|                  3|             307.26|                  102.42|
|            6| 

In [0]:
spark.sql("select * from customers").show()

+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_pincode|
+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|       NULL|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|            NULL|
|          1|       Richard|     Hernandez|     XXXXXXXXX|        XXXXXXXXX|  6303 Heather Plaza|  Brownsville|            TX|           78521|
|          2|          Mary|       Barrett|     XXXXXXXXX|        XXXXXXXXX|9526 Noble Embers...|    Littleton|            CO|           80126|
|          3|           Ann|         Smith|     XXXXXXXXX|        XXXXXXXXX|3422 Blue Pioneer...|       Caguas|            PR|          

In [0]:
customer_schema ="customer_id long,customer_fname STRING,customer_lname STRING,customer_email STRING,customer_password STRING,customer_street STRING,customer_city STRING,customer_state STRING,customer_pincode long"


In [0]:
customers_df =spark.read \
    .schema(customer_schema) \
        .format("csv") \
                  .option("header",True) \
                      .load("dbfs:/FileStore/customersweek17.csv")

In [0]:
customers_df.createOrReplaceTempView("customers")

In [0]:
spark.sql("select * from orders limit 10").show()

+--------+--------------------+-----------+---------------+
|order_id|          order_date|customer_id|   order_status|
+--------+--------------------+-----------+---------------+
|    NULL|          order_date|       NULL|   order_status|
|       1|2013-07-25 00:00:...|      11599|         CLOSED|
|       2|2013-07-25 00:00:...|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|      12111|       COMPLETE|
|       4|2013-07-25 00:00:...|       8827|         CLOSED|
|       5|2013-07-25 00:00:...|      11318|       COMPLETE|
|       6|2013-07-25 00:00:...|       7130|       COMPLETE|
|       7|2013-07-25 00:00:...|       4530|       COMPLETE|
|       8|2013-07-25 00:00:...|       2911|     PROCESSING|
|       9|2013-07-25 00:00:...|       5657|PENDING_PAYMENT|
+--------+--------------------+-----------+---------------+



In [0]:
joined_df = spark.sql("""select orders.order_id,
             order_items.order_item_id,
             orders.customer_id,
             order_items.order_item_product_id,
             order_items.order_item_quantity,
             order_items.order_item_subtotal,
             order_items.order_item_product_price,
             customers.customer_fname,
             customers.customer_lname,
             customers.customer_city,
             customers.customer_state,
             customers.customer_pincode from orders
             join order_items 
             on orders.order_id = order_items.order_item_id
             join customers
             on orders.customer_id = customers.customer_id """)

In [0]:
from pyspark.sql.functions import *

results_df = joined_df.groupBy(
    "order_id",
    "customer_id",
    "customer_fname",
    "customer_lname",
    "customer_city",
    "customer_state",
    "customer_pincode"
).agg(
    collect_list(
        struct(
            "order_item_id",
            "order_item_product_id",
            "order_item_quantity",
            "order_item_product_price",
            "order_item_subtotal"
        )
    ).alias("order_items")
)

results_df.show(truncate=False)


+--------+-----------+--------------+--------------+--------------+--------------+----------------+-------------------------------+
|order_id|customer_id|customer_fname|customer_lname|customer_city |customer_state|customer_pincode|order_items                    |
+--------+-----------+--------------+--------------+--------------+--------------+----------------+-------------------------------+
|8       |2911       |Mary          |Smith         |Caguas        |PR            |725             |[{8, 692, 4, 257.23, 1028.92}] |
|12      |1837       |Mary          |Vega          |Caguas        |PR            |725             |[{12, 411, 4, 19.86, 79.44}]   |
|14      |9842       |Mary          |Smith         |Caguas        |PR            |725             |[{14, 492, 2, 182.13, 364.26}] |
|16      |7276       |Pamela        |Smith         |Caguas        |PR            |725             |[{16, 721, 1, 271.54, 271.54}] |
|23      |4367       |Danielle      |Moran         |Springfield   |MO       

In [0]:
results_df.show(20)

+--------+-----------+--------------+--------------+--------------+--------------+----------------+--------------------+
|order_id|customer_id|customer_fname|customer_lname| customer_city|customer_state|customer_pincode|         order_items|
+--------+-----------+--------------+--------------+--------------+--------------+----------------+--------------------+
|       8|       2911|          Mary|         Smith|        Caguas|            PR|             725|[{8, 692, 4, 257....|
|      12|       1837|          Mary|          Vega|        Caguas|            PR|             725|[{12, 411, 4, 19....|
|      14|       9842|          Mary|         Smith|        Caguas|            PR|             725|[{14, 492, 2, 182...|
|      16|       7276|        Pamela|         Smith|        Caguas|            PR|             725|[{16, 721, 1, 271...|
|      23|       4367|      Danielle|         Moran|   Springfield|            MO|           65807|[{23, 577, 4, 294...|
|      24|      11441|          

In [0]:
results_df \
.repartition(1) \
    .write \
        .format("json") \
            .mode("overwrite") \
                .option("path","dbfs:/FileStore/merge.json") \
                    .save()